*В этой тетрадке находится код подпроекта по получению названия района города Москвы, в котором находится заведение общественного питания из исходного датасета*
## Импорт библиотек

In [ ]:
import pandas as pd
import time
import requests
import warnings
warnings.filterwarnings('ignore')

## Ресурс и авторизационные данные

In [71]:
TOKEN = ''
URL = 'https://geocode-maps.yandex.ru/1.x/?'

## Чтение файла-заготовки

In [72]:
uniq_addresses = pd.read_csv('uniq_addresses.csv')
del uniq_addresses['Unnamed: 0']
uniq_addresses.head()

,uniq_adress,obj_count
0,"город Москва, Ходынский бульвар, дом 4",95
1,"город Москва, Пресненская набережная, дом 2",62
2,"город Москва, проспект Мира, дом 211, корпус 2",60
3,"город Москва, Кировоградская улица, дом 13А",53
4,"город Москва, площадь Киевского Вокзала, дом 2",47


## Функция запроса и получение координат

In [78]:
def get_coordinate(uniq_adress):
    geocode = uniq_adress.encode('UTF-8')
    PARAM = {
        'geocode' : geocode,
        'apikey' : TOKEN,
        'format' : 'json'
    }
    r = requests.get(URL, params = PARAM)
    try:
        coordinate = r.json(
        )['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].replace(' ',',')
    except:
        coordinate = '00.000000,00.000000'
    time.sleep(0.5)
    return coordinate

In [79]:
# применение функции
uniq_addresses['coordinate'] = uniq_addresses['uniq_adress'].apply(get_coordinate)

In [83]:
# запись в файл
uniq_addresses.to_csv('adr_coordinate.csv') 

In [144]:
# просмотр объектов с неопределенными координатами
pd.options.display.max_colwidth = 100
undefined_addresses = uniq_addresses.query('coordinate == "00.000000,00.000000"')
undefined_addresses.head(10)

,uniq_adress,obj_count,coordinate
183,"город Москва, 87-й километр Московской Кольцевой Автодороги, дом 8",7,"00.000000,00.000000"
1801,"город Москва, 19-й километр Московской Кольцевой Автодороги, владение 20, строение 1",2,"00.000000,00.000000"
1949,"город Москва, 82-й километр Московской Кольцевой Автодороги, владение 18",2,"00.000000,00.000000"
2297,"город Москва, 87-й километр Московской Кольцевой Автодороги, владение 14",1,"00.000000,00.000000"
2599,"город Москва, 73-й километр Московской Кольцевой Автодороги, дом 7, корпус 1",1,"00.000000,00.000000"
2911,"город Москва, 73-й километр Московской Кольцевой Автодороги, дом 7",1,"00.000000,00.000000"
2922,"город Москва, 67-й километр Московской Кольцевой Автодороги, дом 7",1,"00.000000,00.000000"
3456,"город Москва, 19-й километр Московской Кольцевой Автодороги, владение 20, строение 2",1,"00.000000,00.000000"
3540,"город Москва, 31-й километр Московской Кольцевой Автодороги, владение 8, строение 1",1,"00.000000,00.000000"
3584,"город Москва, 104-й километр Московской Кольцевой Автодороги, дом 6",1,"00.000000,00.000000"


Геокодер не определил координаты 24 объектов, адрес которых содержит номер километра МКАД. Объекты сильно удалены от центра и могут не учитываться. Количество объектов относительно общего невелико, поэтому значительную погрешность это пренебрежение не внесет.
## Функция запроса и получение названия района
Получение района для тех объектов, координаты которых получены

In [95]:
defined_coordinates = uniq_addresses.query('coordinate != "00.000000,00.000000"')

In [149]:
def get_district(coordinate):
    PARAM = {
        'geocode' : coordinate,
        'apikey' : TOKEN,
        'format' : 'json',
        'kind' : 'district',
        'result' : 1
    }
    r = requests.get(URL, params = PARAM)
    try:
        district = r.json(
        )['response']['GeoObjectCollection']['featureMember'][0][
            'GeoObject']['metaDataProperty']['GeocoderMetaData']['Address']['Components'][5]['name']
    except:
        district = 'not_defined'
    time.sleep(0.1)
    return district   

In [188]:
def get_street(coordinate):
    PARAM = {
        'geocode' : coordinate,
        'apikey' : TOKEN,
        'format' : 'json',
        'kind' : 'street',
        'result' : 1      
    }
    r = requests.get(URL, params = PARAM)
    try:
        street = r.json()['response']['GeoObjectCollection']['featureMember'][0][
            'GeoObject']['metaDataProperty']['GeocoderMetaData']['Address']['Components'][4]['name']
    except:
        street = 'not_defined'
    time.sleep(0.1)
    return street

In [191]:
defined_coordinates['district'] = defined_coordinates['coordinate'].apply(get_district)

In [195]:
defined_coordinates['street'] = defined_coordinates['coordinate'].apply(get_street)

In [197]:
final_addresses_data = defined_coordinates.copy()

In [199]:
final_addresses_data.to_csv('final_addresses.csv') 

In [203]:
final_addresses_data.head(15)

,uniq_adress,obj_count,coordinate,district,street
0,"город Москва, Ходынский бульвар, дом 4",95,"37.531011,55.790458",Хорошёвский район,улица Авиаконструктора Микояна
1,"город Москва, Пресненская набережная, дом 2",62,"37.539644,55.749299",Пресненский район,Выставочный переулок
2,"город Москва, проспект Мира, дом 211, корпус 2",60,"37.662093,55.845855",район Свиблово,проезд Серебрякова
3,"город Москва, Кировоградская улица, дом 13А",53,"37.607026,55.612182",район Чертаново Центральное,Варшавское шоссе (дублёр)
4,"город Москва, площадь Киевского Вокзала, дом 2",47,"37.566072,55.744632",район Дорогомилово,Большая Дорогомиловская улица
5,"город Москва, улица Земляной Вал, дом 33",46,"37.659164,55.757339",Басманный район,улица Земляной Вал
6,"город Москва, Мытная улица, дом 74",46,"37.620555,55.712112",Даниловский район,улица Серпуховский Вал
7,"город Москва, улица Ленинская Слобода, дом 26",41,"37.654601,55.710124",not_defined,улица Ленинская Слобода
8,"город Москва, улица Новый Арбат, дом 21",40,"37.586158,55.75204",район Арбат,улица Новый Арбат
9,"город Москва, Кутузовский проспект, дом 57",40,"37.476169,55.727927",район Фили-Давыдково,Староможайское шоссе


Геокодер при определении улицы возвращает множество вариантов улиц, даже при указании в запросе возвращать один результат поиска. Результат, судя по первым 15 строкам, неудовлетворительный. В проекте предпочтение будет отдано информации об улице, полученной непосредственно из исходного датасета 